# Oferecendo o melhor plano de celular

Neste projeto tenho o objetivo de desenvolver um modelo de apredizado de máquina que possa recomendar um plano de celular para os clientes com base no comportamento de uso do celular dos mesmos. Eu utilizei os dados *fictícios* da empresa Megaline, que oferece dois planos de celular: Smart ou Ultra. Os dados de comportamento dos clientes já foram processados anteriormente. Os dados incluem as seguintes informações: 

- сalls — número de chamadas
- minutes — duração total da chamada em minutos
- messages — número de mensagens de texto
- mb_used — Tráfego de Internet usado em MB
- is_ultra — plano para o mês atual (Ultra - 1, Smart - 0)

## Importando bibliotecas e dados

In [1]:
### importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
### importando dados
try:
    behavior = pd.read_csv('users_behavior_upd.csv')
except:
    behavior = pd.read_csv("/datasets/users_behavior_upd.csv")

## Estudando os dados

In [3]:
behavior.head(10)

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Comentário**: Não existem dados faltantes, e todas as variáveis estão com tipo adequado de dados.

## Separando dados para treino, validação e teste

In [5]:
### separando os features
features = behavior.drop(labels='is_ultimate', axis=1)
features.head(5)

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


In [6]:
### separando o target
target= behavior['is_ultimate']
target.head(5)

0    0
1    0
2    0
3    1
4    0
Name: is_ultimate, dtype: int64

In [7]:
### separando dados de treino
features_train, features_va_test, target_train, target_va_test = train_test_split(features, target, train_size=0.6, random_state= 42)

In [8]:
### separando dados de validação e teste
features_valid, features_test, target_valid, target_test = train_test_split(features_va_test, target_va_test, train_size=0.5, random_state= 42)

**Comentário**: Eu separei 60% dos dados para o conjunto de treino, 20% para o conjunto de validação e outros 20% para o conjunto de teste.

## Treinando diferentes modelos

### Treinando `Logistic regression`

In [9]:
### preparando variáveis que receberão os resultados
best_logis_model = None
best_c_value = 1
c_values = [0.1, 0.01, 0.001]
best_logis_acc = 0

In [10]:
### ajustando modelos logísticos com diferentes hiperparâmetros
for c in c_values:
    logis_model = LogisticRegression(solver= 'lbfgs', C = c, max_iter=1000, random_state=42)
    logis_model.fit(features_train, target_train)
    prediction_valid = logis_model.predict(features_valid)
    logis_acc = accuracy_score(target_valid, prediction_valid)
    if logis_acc > best_logis_acc:
        best_logis_model = logis_model
        best_c_value = c
        best_logis_acc = logis_acc
        print(f'Regularização: {c}, com acurácia de: {round(logis_acc,2)}')

Regularização: 0.1, com acurácia de: 0.72
Regularização: 0.01, com acurácia de: 0.74


**Comentário**: O modelo de `Logistic Regression` atingiu maior acurácia (0.74) com regularização de 0.01.

### Treinando `Decision Tree`

In [11]:
### preparando variáveis que receberão os resultados
best_tree_model = None
best_tree_depth = 0
best_tree_acc = 0

In [12]:
### ajustando modelos de árvores com diferentes hiperparâmetros
for depth in range(1,21):
    tree_model = DecisionTreeClassifier(max_depth= depth, random_state = 42)
    tree_model.fit(features_train, target_train)
    prediction_valid = tree_model.predict(features_valid)
    tree_acc = accuracy_score(target_valid, prediction_valid)
    if tree_acc > best_tree_acc:
        best_tree_model = tree_model
        best_tree_depth = depth
        best_tree_acc = tree_acc
        print(f'Profundidade atualizada: {depth}, com acurária de: {round(tree_acc,2)}')

Profundidade atualizada: 1, com acurária de: 0.73
Profundidade atualizada: 2, com acurária de: 0.78
Profundidade atualizada: 3, com acurária de: 0.79
Profundidade atualizada: 8, com acurária de: 0.8


**Comentário**: O modelo de `Decision Tree` atingiu maior acurácia (0.80) ao gerar uma árvore com 8 nós de profundidade. 

### Treinando `Random Forest`

In [13]:
### preparando variáveis que receberão os resultados
best_forest_model = None
best_forest_est = 0
best_forest_depth = 0
best_forest_acc = 0

In [14]:
### ajustando modelos de floresta com diferentes hiperparâmetros
for est in range(1, 31):
    for depth in range(1,21):
        forest_model = RandomForestClassifier(n_estimators= est, max_depth= depth, random_state = 42)
        forest_model.fit(features_train, target_train)
        prediction_valid = forest_model.predict(features_valid)
        forest_acc = accuracy_score(target_valid, prediction_valid)
        if forest_acc > best_forest_acc:
            best_forest_model = forest_model
            best_forest_depth = depth
            best_forest_acc = forest_acc
            print(f'Número de árvores atualizado: {est}, Profundidade atualizada: {depth}, com acurária de: {round(forest_acc,2)}')

Número de árvores atualizado: 1, Profundidade atualizada: 1, com acurária de: 0.76
Número de árvores atualizado: 1, Profundidade atualizada: 2, com acurária de: 0.79
Número de árvores atualizado: 1, Profundidade atualizada: 4, com acurária de: 0.8
Número de árvores atualizado: 1, Profundidade atualizada: 7, com acurária de: 0.8
Número de árvores atualizado: 3, Profundidade atualizada: 10, com acurária de: 0.8
Número de árvores atualizado: 6, Profundidade atualizada: 6, com acurária de: 0.8
Número de árvores atualizado: 7, Profundidade atualizada: 6, com acurária de: 0.81
Número de árvores atualizado: 7, Profundidade atualizada: 9, com acurária de: 0.81
Número de árvores atualizado: 7, Profundidade atualizada: 12, com acurária de: 0.81
Número de árvores atualizado: 22, Profundidade atualizada: 9, com acurária de: 0.82
Número de árvores atualizado: 24, Profundidade atualizada: 9, com acurária de: 0.82


**Comentário**: o modelo de `Random Forest` atingiu maior acurácia (0.82) ao gerar 24 árvores, cada uma com 9 nós de profundidade. 

## Testando os modelos

In [15]:
### testando modelo Logistic Regression
logis_prediction_test = best_logis_model.predict(features_test)
logis_acc_test = accuracy_score(target_test, logis_prediction_test)

In [16]:
### testando modelo Decision Tree
tree_prediction_test = best_tree_model.predict(features_test)
tree_acc_test = accuracy_score(target_test, tree_prediction_test)

In [17]:
### testando modelo Decision Tree
forest_prediction_test = best_forest_model.predict(features_test)
forest_acc_test = accuracy_score(target_test, forest_prediction_test)

In [18]:
print(f'Acurácia Logistic Regression: {round(logis_acc_test,3)}')
print(f'Acurácia Decision Tree: {round(tree_acc_test, 3)}')
print(f'Acurácia Random Forest: {round(forest_acc_test,3)}')

Acurácia Logistic Regression: 0.768
Acurácia Decision Tree: 0.798
Acurácia Random Forest: 0.816


**Comentário**: O modelo `Random Forest`atingiu a maior acurária durante o teste de modelos, com valor de 0.816. Este foi seguido pelos modelos `Decision Tree` e `Logistic Regression`, respectivamente. 

## Conclusão

Neste projeto eu desenvolvi modelos de aprendizado de máquina para oferecer o melhor plano de celular para os clientes com base no comportamento de uso dos celular dos mesmos. Eu treinei três tipos de modelo: `Logistic Regression`, `Decision Tree` e `Random Forest`. Entre estes, `Random Forest`atingiu a maior acurácia, permitindo atribuir um plano de celular condizente ao comportamento do cliente em cerca de 80% dos casos.